<a href="https://colab.research.google.com/github/jazoza/cultural-data-analysis/blob/main/03_CDA_HH_dataset_representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cultural Data Analysis

Introduction to working with datasets

In [ ]:
# import necessary libraries
import os, re, csv
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from itertools import islice

## Loading the datasets: heritage homes webistes

The dataset is stored in a shared google drive:
https://drive.google.com/drive/folders/11Shm0edDOiWrOe56fzJQRZi-v_BPSW8E?usp=drive_link

Add it to your drive.

To access it, load your gdrive in 'Files' (see left pane of the notebook in google colab) and navigate to the shared folder. You may need to click on 'refresh' to make it appear on the list.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### A. Working witn one country

In [ ]:
# Country code: change here between 'NL' and 'UK'
cc = 'NL'

In [ ]:
gdrive_path = '/content/gdrive/MyDrive/CDA/'

In [ ]:
raw_data_file = gdrive_path+cc+'_dataset_website-content-crawler.json'

In [ ]:
# Import json data from Aipfy scraping
df=pd.read_json(raw_data_file)

# Print the DataFrame
df.head()

In [ ]:
# check if there are further datasets to add per country

!ls "$gdrive_path" | grep "$cc"

In [ ]:
df_missing1 = pd.read_json(gdrive_path+'/NL_LG_dataset_website-content-crawler_2025-02-06_09-40-33-880.json')
result = pd.concat([df, df_missing1])
df = result
df.head()

In [ ]:
# select only two columns for analysis: url and text
df=df[['url','text']]
df.head()

Join all pages from a domain to an entry in the analysis. To do this, add a new column which will contain only the main domain name.

In [ ]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?:\\w+\\.)*\\w+\\.\\w*', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [ ]:
# Load the list of domains from a csv file:
cc_column = cc+' domains'
#print(cc_column)

urls = pd.read_csv(gdrive_path+'url_lists/'+cc+'_urls.csv')[cc_column].values.tolist()

# Extract main domains from nl_urls
domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}

# Check if main domains in list_of_links match any domain in nl_domains
matching_links = [link for link in df.url if extract_main_domain(link) in domains]

In [ ]:
print(len(matching_links))

In [ ]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column
df['domain'] = df['url'].apply(extract_main_domain)
df.head()

### B. Working with all countries

In [ ]:
# Country code: change here between 'NL' and 'UK'
cc_list = ['NL', 'UK', 'DE', 'FR']

In [ ]:
gdrive_path = '/content/gdrive/MyDrive/CDA/'

In [ ]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?:\\w+\\.)*\\w+\\.\\w*', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [ ]:
# Import scraped json data into 4 separate dataframes
df0=pd.read_json(gdrive_path+cc_list[0]+'_dataset_website-content-crawler.json')
# select only two columns for analysis: url and text
df0=df0[['url','text']]

df1=pd.read_json(gdrive_path+cc_list[1]+'_dataset_website-content-crawler.json')
# select only two columns for analysis: url and text
df1=df1[['url','text']]

df2=pd.read_json(gdrive_path+cc_list[2]+'_dataset_website-content-crawler.json')
# select only two columns for analysis: url and text
df2=df2[['url','text']]

df3=pd.read_json(gdrive_path+cc_list[3]+'_dataset_website-content-crawler.json')
# select only two columns for analysis: url and text
df3=df3[['url','text']]

df0.head()


Join all pages from a domain to an entry in the analysis. To do this, add a new column which will contain only the main domain name.

In [ ]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column

# first, create a mapping of dataframes which could be addressed in a loop
df_dict = {'0':df0, '1':df1, '2':df2, '3':df3}

# then, loop through the df_dict to update each dataframe
for k, v in df_dict.items():
  cc_column = cc_list[int(k[-1])]+' domains'
  cc = cc_list[int(k[-1])]
  # print(cc_column, cc)
  urls = pd.read_csv(gdrive_path+'url_lists/'+cc_list[int(k[-1])]+'_urls.csv')[cc_column].values.tolist()
  domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}
  matching_links = [link for link in v.url if extract_main_domain(link) in domains]
  # update the dataframe
  v['domain'] = v['url'].apply(extract_main_domain)

# check one of the dataframes
df1.head()

## Prepare the analysis

In [ ]:
# Get the stopwords list for one language (defined in cc)
# Redefine cc as necessary
cc = 'NL'
# Define the path to the file in github
stopwords_cc = "https://raw.githubusercontent.com/jazoza/cultural-data-analysis/refs/heads/main/stopwords_archive/"+cc+".txt"
# Download the file, it should appear under 'sample_data' folder
!wget "$stopwords_cc"

In [ ]:
# load a list of 'stopwords' in the language you are analyzing
def get_stopwords_list(stop_file_path):
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))
stopwords_path = cc+".txt"
stopwords = get_stopwords_list(stopwords_path)

In [ ]:
# you may need to include additional words which you notice as too frequent
special_stop_words = ['nbsp', 'nl', 'the', 'and', 'und', 'we', 'to']
stopwords_ext = stopwords+special_stop_words

## 1. Term frequency

The cells below will compute a term-matrix and calculate the frequency of each unique word (token) in the corpus

This can be done for ALL words in the corpus, or ALL MEANINGFUL words (without so-called stop-words like 'the' or 'het')

In [ ]:
# CALCULATE TERM FREQUENCY OF ALL TERMS

from sklearn.feature_extraction.text import CountVectorizer
# convert the text documents into a matrix of token (word) counts
cvec_all = CountVectorizer().fit(df.text)
df_matrix_all = cvec_all.transform(df.text)
df_all = np.sum(df_matrix_all,axis=0)
terms = np.squeeze(np.asarray(df_all))
# print the 'shape' of the matrix - it should indicate the number of unique terms
print(terms.shape)
term_freq_df_all = pd.DataFrame([terms],columns=cvec_all.get_feature_names_out()).transpose() #term_freq_df is with stopwords
term_freq_df_all.columns = ['terms']
# show the first ten words [:10];
# change the values in the brackets to show 30th-40th words [30:40]
# or show the last ten words [:-10]
term_freq_df_all.sort_values(by='terms', ascending=False).iloc[:-10]

In [ ]:
# CALCULATE TERM FREQUENCY WITHOUT STOP-WORDS

cvec_stopped = CountVectorizer(stop_words=stopwords_ext, token_pattern=r'(?u)\b[A-Za-z]{2,}\b') # see above, import frozenset from stopwords_archive in correct language) # see above, import frozenset from stopwords_archive in correct language
cvec_stopped.fit(df.text)
document_matrix = cvec_stopped.transform(df.text)
term_batches = np.linspace(0,document_matrix.shape[0],10).astype(int)
i=0
df_stopped = []
while i < len(term_batches)-1:
    batch_result = np.sum(document_matrix[term_batches[i]:term_batches[i+1]].toarray(),axis=0)
    df_stopped.append(batch_result)
    print(term_batches[i+1],"entries' term frequency calculated")
    i += 1

terms_stopped = np.sum(df_stopped,axis=0)
print(terms_stopped.shape)
term_freq_df_stopped = pd.DataFrame([terms_stopped],columns=cvec_stopped.get_feature_names_out()).transpose()
term_freq_df_stopped.columns = ['terms']
term_freq_df_stopped.sort_values(by='terms', ascending=False).iloc[:10]


### 1.1 TF-IDF vectorization

- What is TF/IDF (term frequency / inverse document frequency)? https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords_ext)
# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(df['text'])
# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
# Add filenames as index
tfidf_df.index = df['domain']
# Print the TF-IDF DataFrame
tfidf_df.head()

In [ ]:
# Add column for document frequency aka number of times word appears in all documents
tfidf_df.loc['ALL'] = (tfidf_df > 0).sum()

In [ ]:
tfidf_df.head() # first five rows

In [ ]:
# 10 most frequent words!

tfidf_slice = tfidf_df[term_freq_df_stopped.sort_values(by='terms', ascending=False).iloc[:10].index.tolist()]
tfidf_slice.sort_index().round(decimals=2).head() # first five rows

In [ ]:
# reorganize the DataFrame so that the words are in rows rather than columns
tfidf_df = tfidf_df.drop('ALL', errors='ignore')
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df.head()

In [ ]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'domain': 'document','level_1': 'term'})
tfidf_df.head()

In [ ]:
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head

In [ ]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

# Terms in this list will get a red dot in the visualization
term_list = ['kasteel', 'huis']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)

### 2. Collocations

### 2.1 Analyze specific collocations

In [ ]:
# search for words from this list or use another list
search_words = ['royal']

In [ ]:
# SCI-KIT method, produces lists of co-occurencies for specific terms
def vectorize_text(df):
    vectorizer = text.CountVectorizer()
    X = vectorizer.fit_transform(df['text'])
    return X, vectorizer

def find_collocations(text, target_words):
    words = text.split()
    collocations = []
    for i in range(len(words) - 1):
        if words[i] in target_words:
            collocations.append((words[i], words[i + 1]))
        if words[i + 1] in target_words:
            collocations.append((words[i + 1], words[i]))
    return collocations

def get_frequent_collocations(df, most_frequent_words):
    collocations = []
    for text in df['text']:
        collocations.extend(find_collocations(text, most_frequent_words))
    collocation_counts = Counter(collocations)
    frequent_collocations = {}
    for word in most_frequent_words:
        word_collocations = {collocation: count for collocation, count in collocation_counts.items() if word in collocation}
        frequent_collocations[word] = dict(islice(Counter(word_collocations).most_common(20), 20))
    return frequent_collocations

def analyze_word_collocations(df):
    X, vectorizer = vectorize_text(df)
    most_frequent_words = search_words
    frequent_collocations = get_frequent_collocations(df, most_frequent_words)
    return frequent_collocations

In [ ]:
collocations = analyze_word_collocations(df)

In [ ]:
data = []
for word, colloc_dict in collocations.items():
   for collocation, count in colloc_dict.items():
       #collocation_str = ' '.join(collocation)  # Join collocation words into a single string
       data.append([word, collocation[1], count])
collocations_df = pd.DataFrame(data, columns=['Word', 'Collocation', 'Count'])
print(collocations_df.to_markdown(index=True))

### 3. Analyse collocations in sentences

In [ ]:
#function to remove non-ascii characters
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [ ]:
# import the advanced Natural Language Processing (NLP) library
# which we will use to analze the grammatical structure
import spacy

In [ ]:
# download the suitable language pipeline
# Dutch: nl_core_news_sm
# French: nl_core_news_sm
# German: nl_core_news_sm
# English is available by default
!python -m spacy download nl_core_news_sm

In [ ]:
nlp = spacy.load('nl_core_news_sm')

In [ ]:
import string

#function to clean and lemmatize comments
def clean_documents(text):
    #remove punctuations
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", str(text))
    #use spacy to lemmatize comments
    doc = nlp(nopunct, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma

In [ ]:
#apply function to clean and lemmatize comments
lemmatized = df.text.map(clean_documents)
#make sure to lowercase everything
lemmatized = lemmatized.map(lambda x: [word.lower() for word in x])
lemmatized.head()

In [ ]:
unlist_documents = [item for items in lemmatized for item in items]

In [ ]:
# You would use these commands to save lemmatized text into a 'pickle' for later use
# The current setup does not enable you to overwrite existing files in the CDA/jar folder,
# so you would have to save the 'pickle' files elsewhere (for example in sample_data folder)
# If you want to reactivate this code, remove the tripple quotes ''' from the beginning and end
'''
# save these outputs for later
with open(gdrive_path+'jar/lemmatized.pickle', 'wb') as handle_l:
    pickle.dump(lemmatized, handle_l, protocol=pickle.HIGHEST_PROTOCOL)

with open(gdrive_path+'jar/unlist_documents.pickle', 'wb') as handle_u:
    pickle.dump(unlist_documents, handle_u, protocol=pickle.HIGHEST_PROTOCOL)
  '''

In [ ]:
# load saved pickles
with open(gdrive_path+'jar/'+cc+'_lemmatized.pickle', 'rb') as handle_l:
    lemmatized = pickle.load(handle_l)

with open(gdrive_path+'jar/'+cc+'_unlist_documents.pickle', 'rb') as handle_u:
    unlist_documents = pickle.load(handle_u)

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
# initiate bigrams and trigrams
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

In [ ]:
# identify all collocations in the flat list of words from all documents
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_documents)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_documents)

Calculate basic frequency

In [ ]:
bigram_freq = bigramFinder.ngram_fd.items()

In [ ]:
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
bigramFreqTable.head().reset_index(drop=True)

In [ ]:
# compute basic trigrams frequency
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
trigramFreqTable[:10]

Find meaningful bi- and tri-grams by filtering adjectives and nouns based on an nltk functionality

In [ ]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    for word in ngram:
        if word in stopwords_ext:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filtered_bi[:10]

Use advanced statistical methods like the Chi-Square to identify meaninful collocations
https://en.wikipedia.org/wiki/Chi-squared_test

In [ ]:
# filter bigrams using chi-square
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
bigramChiTable.head()

In [ ]:
# find meaningful trigrams by filtering basic frequency table
# function to filter trigrams
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in stopwords_ext:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [ ]:
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]
filtered_tri[:10]

In [ ]:
# Chi-sqare frequency calculation for trigrams
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
trigramChiTable.head(20)